In [5]:
import numpy as np
import pandas as pd
from pulp import *
from geopy.distance import distance
import random
import pickle

In [14]:
wells = pd.read_csv("data/wells/AllWells_20210915.csv")

# Oil & Gas, Dry Gas (no liquids), Gas and "Liquefied Gas"
wells_oilgas = wells[ (wells['WellType'] == 'OG')  |
                      (wells['WellType'] == 'DG')  |
                      (wells['WellType'] == 'GAS') |
                      (wells['WellType'] == 'LG') ]
wells_latlon = wells_oilgas[['Latitude', 'Longitude']].to_records(index=False)
wells_ids = wells_oilgas['API'].tolist()

In [15]:
print(wells_oilgas.WellType.value_counts())
wells_oilgas.head()

OG     158064
DG       5003
GAS      1417
LG          6
Name: WellType, dtype: int64


,OID,API,LeaseName,WellNumber,WellStatus,WellType,WellTypeLa,OperatorCo,OperatorNa,FieldName,...,Township,Range,BaseMeridi,Latitude,Longitude,GISSource,isConfiden,isDirectio,SpudDate,WellSymbol
1,-1,402100016,Afton Community 1,1,Plugged,DG,Dry Gas,V1150,"Venoco, LLC",Afton Gas,...,19N,01W,MD,39.451515,-121.950668,hud,N,N,12/02/1943,PluggedDG
4,-1,402100017,Afton Community 2,1,Plugged,DG,Dry Gas,V1150,"Venoco, LLC",Afton Gas,...,19N,01W,MD,39.460217,-121.953537,hud,N,N,04/14/1945,PluggedDG
5,-1,402120283,Afton Community 2,2,Plugged,DG,Dry Gas,V1150,"Venoco, LLC",Afton Gas,...,19N,01W,MD,39.460033,-121.951424,hud,N,N,10/10/1984,PluggedDG
8,-1,402120815,Anacapa,3-4,Idle,DG,Dry Gas,R4085,"Royale Energy, Inc.",Afton Gas,...,18N,01W,MD,39.443230,-121.951042,GPS,N,N,11/06/2002,IdleDG
9,-1,402120494,Andreotti,15-26,Plugged,DG,Dry Gas,W1700,Western Continental Operating Company,Afton Gas,...,19N,01W,MD,39.470551,-121.947769,GPS,N,N,10/05/1987,PluggedDG


In [19]:
emissions = pd.read_csv(r'./data/Emissions by Unit and Fuel Type - csv/UNIT_DATA-Table 1.csv')
emsCA2019 = emissions[emissions['Reporting Year'] == 2019]
facilities = pd.read_excel(r'./data/ghgp_data_2019.xlsx',
                          sheet_name='Direct Emitters', skiprows=3)
# One Michegan emitter lists its CA headquarters. Retag it as Michigan
facilities.loc[facilities['Facility Id'] == 1000594, 'State'] = 'MI'

facilities = facilities[facilities.State == 'CA']

In [22]:
facilities.shape

(382, 66)

In [20]:
facilitiesB = facilities.rename(inplace=False,
                                columns={'Facility Name':'facility_name',
                                         'City':'city',
                                         'Primary NAICS Code':'naics_code',
                                         'Industry Type (subparts)':'industry_subparts',
                                         'Industry Type (sectors)':'industry_sectors'})

emsCA2019loc = pd.merge(emsCA2019, facilitiesB[['Facility Id',
#                                               'Facility Name2',
                                               'city',
                                               'Zip Code',
                                               'Address',
                                               'County',
                                               'Latitude',
                                               'Longitude',
#                                               'Primary NAICS Code2',
                                               'industry_subparts',
                                               'industry_sectors']],
                        how='left', on='Facility Id')
emsCA2019locagg = emsCA2019loc[['Facility Id','Latitude','Longitude',
                                'Unit CO2 emissions (non-biogenic)']].dropna().groupby(
                                    'Facility Id').agg({'Latitude':'min',
                                                        'Longitude':'min',
                                                        'Unit CO2 emissions (non-biogenic)':'sum'})
emsCA2019locagg.reset_index(inplace=True)
emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
                                                  'Latitude':'lat',
                                                  'Longitude':'lon',
                                                  'Unit CO2 emissions (non-biogenic)':'annual_emissions'})
emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['annual_emissions'] > 0]
emsCA2019locagg.head()


,facility_id,lat,lon,annual_emissions
0,1000005,36.923750,-120.104120,60760.1
2,1000051,37.796500,-121.604900,52901.5
3,1000064,35.447778,-119.260000,64051.2
4,1000066,37.074790,-120.877445,69721.9
5,1000070,34.595300,-117.364700,1357906.4


In [23]:
facility_latlon = emsCA2019locagg[['lat', 'lon']].to_records(index=False)
facility_ids = emsCA2019locagg['facility_id'].tolist()

In [26]:
facility_latlon.shape

(288,)

In [ ]:
# This cell takes ages to run - it's creating a full cartesian matrix of distances between facilities and wells
x = time()
distance_matrix = []
for i in facility_latlon:
    distances = []
    for j in wells_latlon:
        dist = distance(i,j).miles
        distances.append(dist)
    distance_matrix.append(distances)

print(time() - x, 'seconds')

In [ ]:
# This cell takes ages to run - it's creating a full cartesian matrix of distances between facilities and wells
x = time()
distance_matrix2 = np.empty(dtype=float,
                           shape=[len(facility_latlon, len(wells_latlon)])
for i in range(len(facility_latlon)):
    for j in range(len(wells_latlon)):
        distance_matrix2(i,j) = distance(facility_latlon[i],
                                         wells_latlon[j]).miles
print(time() - x, 'seconds')

In [ ]:
# Generate random numbers for demand volumes, to be changed
well_volumes = [random.uniform(100,1000) for i in wells_ids]

In [ ]:
prob = LpProblem('Unbalaced Transportation Problem', LpMinimize)
n_receivers = len(wells_ids)
n_suppliers = len(facility_ids)
supply = emsCA2019locagg['annual_emissions'].tolist()
demand = well_volumes
costs = distance_matrix

In [ ]:
# Takes a long time to run because it's solving the problem
routes = [(i, j) for i in range(n_suppliers) for j in range(n_receivers)]

x = LpVariable.dicts('X', routes, lowBound=0)
prob += lpSum([x[i, j] * costs[i][j] for i in range(n_suppliers) for j in range(n_receivers)])

for i in range(n_suppliers):
    prob += lpSum([x[i, j] for j in range(n_receivers)]) == supply[i]
    
for j in range(n_receivers):
    prob += lpSum([x[i, j] for i in range(n_suppliers)]) <= demand[j]
    
# Solving problem
prob.solve()


In [157]:
# Also takes forever to run

print('Status', LpStatus[prob.status])

print('Z = {}'.format(value(prob.objective)))

Status Optimal
Z = 5912245318.111822


In [158]:

for v in prob.variables():
    if (v.varValue > 0):
        print('{} = {}'.format(v.name, v.varValue))

X_(0,_136084) = 363.79758
X_(0,_136086) = 955.40343
X_(0,_136088) = 818.13738
X_(0,_136090) = 240.09383
X_(0,_136091) = 158.00688
X_(0,_136092) = 507.61208
X_(0,_136093) = 713.27267
X_(0,_136095) = 424.10847
X_(0,_136097) = 277.99646
X_(0,_136099) = 433.44914
X_(0,_136101) = 852.66278
X_(0,_136106) = 884.51993
X_(0,_136107) = 237.23657
X_(0,_136108) = 557.20702
X_(0,_136109) = 403.44382
X_(0,_136112) = 646.59709
X_(0,_136113) = 234.19629
X_(0,_136114) = 634.53243
X_(0,_136115) = 333.57676
X_(0,_136116) = 652.61565
X_(0,_136117) = 133.69395
X_(0,_136118) = 866.56851
X_(0,_136119) = 196.67041
X_(0,_136120) = 797.33213
X_(0,_136121) = 669.63862
X_(0,_136122) = 124.75864
X_(0,_136123) = 353.69293
X_(0,_136124) = 979.89389
X_(0,_136125) = 876.62849
X_(0,_136126) = 776.48752
X_(0,_136127) = 632.61654
X_(0,_136128) = 108.11818
X_(0,_136141) = 395.41373
X_(0,_136142) = 558.73323
X_(0,_136143) = 299.00985
X_(0,_136144) = 550.89054
X_(0,_136149) = 738.29037
X_(0,_136150) = 421.07585
X_(0,_136151

X_(102,_154656) = 364.06659
X_(102,_154658) = 930.27978
X_(102,_154659) = 429.71879
X_(102,_154664) = 395.76255
X_(102,_154666) = 408.94409
X_(102,_154667) = 892.39142
X_(102,_154669) = 302.14906
X_(102,_154671) = 486.92345
X_(102,_154672) = 512.86944
X_(102,_154673) = 420.44798
X_(102,_154674) = 926.5982
X_(102,_154675) = 476.74186
X_(102,_154676) = 700.94441
X_(102,_154677) = 451.06093
X_(102,_154678) = 745.19897
X_(102,_154680) = 534.35921
X_(102,_154681) = 178.39903
X_(102,_154683) = 935.42211
X_(102,_154685) = 717.70113
X_(102,_154687) = 315.04715
X_(102,_154689) = 444.41351
X_(102,_154691) = 551.3183
X_(102,_154692) = 668.75651
X_(102,_154693) = 535.62381
X_(102,_154694) = 929.84817
X_(102,_154695) = 419.80192
X_(102,_154754) = 974.49868
X_(102,_154755) = 701.51846
X_(102,_154771) = 925.83065
X_(102,_154772) = 104.28166
X_(102,_154773) = 397.86186
X_(102,_154774) = 442.42143
X_(102,_154775) = 449.91749
X_(102,_154798) = 662.94575
X_(102,_154806) = 729.18096
X_(102,_154807) = 706.

X_(104,_70029) = 815.34707
X_(104,_70059) = 170.53009
X_(104,_70060) = 747.67853
X_(104,_70193) = 269.51905
X_(104,_70198) = 267.46812
X_(104,_70311) = 371.77776
X_(104,_70312) = 650.42349
X_(104,_71660) = 790.21266
X_(104,_71665) = 714.64609
X_(104,_71670) = 475.68859
X_(104,_71681) = 138.78632
X_(104,_71704) = 252.87295
X_(104,_71718) = 613.55553
X_(104,_71765) = 978.82188
X_(104,_71766) = 349.07186
X_(105,_61640) = 140.63912
X_(105,_61644) = 686.36272
X_(105,_61657) = 569.7081
X_(105,_61675) = 330.32457
X_(105,_61688) = 469.67772
X_(105,_61689) = 437.09074
X_(105,_61690) = 507.74436
X_(105,_61691) = 619.82952
X_(105,_61692) = 498.56007
X_(105,_61716) = 933.73533
X_(105,_61719) = 723.36697
X_(105,_61720) = 964.7375
X_(105,_61740) = 787.51865
X_(105,_61742) = 920.29172
X_(105,_61743) = 169.57241
X_(105,_61744) = 292.75572
X_(105,_61745) = 948.89846
X_(105,_61747) = 157.5004
X_(105,_61757) = 332.69936
X_(105,_61758) = 163.89649
X_(105,_61759) = 593.51316
X_(105,_61760) = 122.41219
X_(1

X_(109,_63741) = 790.06933
X_(109,_63761) = 333.53265
X_(109,_63839) = 939.22301
X_(109,_63862) = 557.18802
X_(109,_63867) = 201.13486
X_(109,_64002) = 439.32176
X_(109,_64051) = 607.85519
X_(109,_64053) = 290.37472
X_(109,_64103) = 265.07902
X_(109,_64104) = 953.75143
X_(109,_64213) = 406.63294
X_(109,_64221) = 945.34911
X_(109,_64271) = 320.73698
X_(109,_64284) = 499.03279
X_(109,_64327) = 910.94082
X_(109,_64328) = 229.05296
X_(109,_64365) = 196.28859
X_(109,_64387) = 918.94293
X_(109,_64391) = 241.57076
X_(109,_64402) = 565.77916
X_(109,_64448) = 317.61359
X_(109,_64517) = 879.03766
X_(109,_64671) = 519.08191
X_(109,_64672) = 704.13682
X_(109,_64741) = 756.90393
X_(109,_64742) = 389.49566
X_(109,_64751) = 158.82115
X_(109,_64756) = 781.73266
X_(109,_64758) = 859.96771
X_(109,_64759) = 191.30096
X_(109,_64785) = 207.61442
X_(109,_64980) = 156.12335
X_(109,_65023) = 972.27536
X_(109,_65024) = 317.40648
X_(109,_65025) = 542.96328
X_(109,_65026) = 628.4124
X_(109,_65027) = 860.7846
X_(

X_(111,_61560) = 886.10344
X_(111,_61602) = 414.03091
X_(111,_61606) = 727.56838
X_(111,_69000) = 819.1226
X_(111,_69001) = 836.1947
X_(111,_69593) = 51.470991
X_(111,_69812) = 288.38394
X_(111,_70741) = 136.92158
X_(112,_132599) = 83.813235
X_(112,_133705) = 781.90445
X_(112,_136994) = 329.8607
X_(112,_61604) = 698.72015
X_(112,_69223) = 927.61797
X_(112,_69390) = 703.12652
X_(112,_69505) = 905.06057
X_(112,_69561) = 362.52463
X_(112,_70479) = 228.57177
X_(113,_70412) = 764.80934
X_(113,_70421) = 767.37976
X_(113,_70424) = 411.32587
X_(113,_70437) = 688.37192
X_(113,_70438) = 946.39159
X_(113,_70439) = 164.87166
X_(113,_70440) = 431.49675
X_(113,_70455) = 374.96838
X_(113,_70458) = 501.72004
X_(113,_70462) = 292.83379
X_(113,_70463) = 181.23448
X_(113,_70464) = 527.17368
X_(113,_70469) = 599.2736
X_(113,_70470) = 677.21813
X_(113,_70471) = 312.65016
X_(113,_70472) = 758.93505
X_(113,_70473) = 241.87055
X_(113,_70474) = 66.189794
X_(113,_70475) = 202.08546
X_(114,_70422) = 292.11131
X_

...

X_(82,_29835) = 433.80972
X_(82,_29860) = 626.12878
X_(82,_29861) = 896.87352
X_(82,_29862) = 302.03797
X_(82,_29863) = 556.174
X_(82,_29864) = 126.21352
X_(82,_29865) = 421.64963
X_(82,_29866) = 783.43547
X_(82,_29867) = 909.34439
X_(82,_29868) = 392.52904
X_(82,_29869) = 307.00153
X_(82,_29871) = 115.95889
X_(82,_29872) = 816.45197
X_(82,_29873) = 257.18163
X_(82,_29874) = 496.19983
X_(82,_29877) = 671.14652
X_(82,_29878) = 171.61872
X_(82,_29880) = 728.44102
X_(82,_29882) = 563.61719
X_(82,_29883) = 727.36676
X_(82,_29885) = 865.18345
X_(82,_29886) = 669.71454
X_(82,_29888) = 989.75563
X_(82,_37899) = 154.33546
X_(82,_37900) = 175.68489
X_(82,_37901) = 361.89158
X_(82,_37902) = 720.41405
X_(82,_37903) = 631.55534
X_(82,_37904) = 113.51622
X_(82,_37905) = 881.39218
X_(82,_37906) = 250.57414
X_(82,_37907) = 432.20964
X_(82,_37933) = 958.2209
X_(82,_37934) = 190.38821
X_(82,_37935) = 293.26845
X_(82,_37936) = 245.21461
X_(82,_67170) = 450.75987
X_(82,_67619) = 829.83613
X_(83,_1147) = 

X_(85,_55472) = 494.42063
X_(85,_55473) = 247.3423
X_(85,_55474) = 209.37129
X_(85,_55475) = 763.44341
X_(85,_616) = 641.00211
X_(85,_625) = 698.83034
X_(85,_63645) = 243.12876
X_(85,_689) = 558.58024
X_(85,_690) = 997.75318
X_(85,_917) = 524.80285
X_(85,_977) = 548.82812
X_(86,_129874) = 150.88218
X_(86,_129875) = 175.77981
X_(86,_135470) = 895.36189
X_(86,_135471) = 234.60651
X_(86,_135472) = 222.71485
X_(86,_135473) = 224.65453
X_(86,_135476) = 190.40668
X_(86,_135477) = 922.15594
X_(86,_135478) = 983.67011
X_(86,_135479) = 290.93456
X_(86,_135480) = 907.57083
X_(86,_135481) = 898.31607
X_(86,_135482) = 277.70588
X_(86,_135483) = 540.48535
X_(86,_135484) = 557.68366
X_(86,_135485) = 177.60024
X_(86,_135489) = 925.71828
X_(86,_135490) = 165.24952
X_(86,_135491) = 599.89661
X_(86,_135492) = 878.37932
X_(86,_135493) = 641.59626
X_(86,_135494) = 165.75536
X_(86,_135495) = 280.41466
X_(86,_135496) = 217.7972
X_(86,_135497) = 953.83913
X_(86,_135498) = 222.83946
X_(86,_135499) = 568.69447

X_(89,_1441) = 275.61988
X_(89,_52961) = 775.02021
X_(89,_85224) = 331.70005
X_(89,_85439) = 550.22338
X_(89,_85556) = 756.61721
X_(89,_85596) = 637.67151
X_(89,_85625) = 491.34952
X_(89,_86497) = 232.19269
X_(89,_86698) = 903.68251
X_(89,_86721) = 644.87075
X_(89,_86774) = 773.27837
X_(89,_86780) = 255.66028
X_(89,_86788) = 866.15311
X_(89,_86802) = 416.15937
X_(89,_86803) = 229.89694
X_(89,_86834) = 310.0013
X_(89,_86860) = 671.46175
X_(89,_86885) = 936.24731
X_(89,_86890) = 708.34057
X_(89,_86902) = 302.3379
X_(89,_86953) = 288.33153
X_(89,_86986) = 748.25598
X_(89,_87062) = 518.76834
X_(89,_87076) = 802.71708
X_(89,_87392) = 476.3257
X_(89,_87465) = 941.00393
X_(89,_87466) = 475.79561
X_(89,_87473) = 580.99954
X_(89,_87584) = 430.98964
X_(89,_87749) = 865.0606
X_(89,_87753) = 687.38587
X_(89,_87755) = 688.64086
X_(89,_87843) = 915.46599
X_(89,_87883) = 192.79624
X_(89,_87891) = 548.3849
X_(89,_87893) = 292.61247
X_(89,_87902) = 581.85454
X_(89,_87921) = 858.45907
X_(89,_87973) = 18

X_(94,_85218) = 577.99888
X_(94,_86005) = 130.5628
X_(94,_86030) = 298.34857
X_(94,_86038) = 877.82104
X_(94,_86045) = 892.12704
X_(94,_86159) = 500.14659
X_(94,_86160) = 306.31784
X_(94,_86161) = 682.09335
X_(94,_86168) = 180.19366
X_(94,_86332) = 846.99401
X_(94,_86371) = 873.05413
X_(94,_86407) = 548.32352
X_(94,_86418) = 877.15757
X_(94,_86420) = 788.28984
X_(94,_86421) = 318.18549
X_(94,_86450) = 376.3032
X_(94,_86452) = 431.65729
X_(94,_86766) = 548.27443
X_(94,_86793) = 726.43826
X_(94,_86813) = 118.64131
X_(94,_86842) = 244.66195
X_(94,_86848) = 769.62091
X_(94,_86855) = 134.04066
X_(94,_86867) = 643.41884
X_(94,_86931) = 319.20426
X_(94,_86990) = 704.21585
X_(94,_87032) = 259.30467
X_(94,_87035) = 738.61254
X_(94,_87069) = 294.80079
X_(94,_87086) = 854.55023
X_(94,_87105) = 798.7871
X_(94,_87122) = 562.10219
X_(94,_87152) = 631.45585
X_(94,_87282) = 116.94752
X_(94,_87325) = 283.07124
X_(94,_87402) = 742.52964
X_(94,_87412) = 311.64522
X_(94,_87483) = 547.08501
X_(94,_87634) =

X_(98,_97988) = 480.69839
X_(98,_97989) = 555.59061
X_(98,_97996) = 923.65598
X_(98,_98002) = 969.01561
X_(98,_98004) = 544.3318
X_(98,_98007) = 271.24981
X_(98,_98009) = 888.01606
X_(98,_98010) = 382.25628
X_(98,_98011) = 372.68163
X_(98,_98012) = 387.5674
X_(98,_98013) = 491.90236
X_(98,_98014) = 484.38834
X_(98,_98015) = 325.32436
X_(98,_98018) = 825.23359
X_(98,_98019) = 128.24206
X_(98,_98020) = 546.43434
X_(98,_98025) = 342.69219
X_(98,_98026) = 459.53991
X_(98,_98027) = 781.22348
X_(98,_98028) = 153.97044
X_(98,_98033) = 965.97233
X_(98,_98038) = 618.93488
X_(98,_98043) = 466.63294
X_(98,_98044) = 994.71719
X_(98,_98045) = 837.20472
X_(98,_98046) = 852.42168
X_(98,_98048) = 887.90438
X_(98,_98052) = 654.57508
X_(98,_98058) = 135.06569
X_(98,_98061) = 732.43946
X_(98,_98062) = 172.51925
X_(98,_98073) = 424.9749
X_(98,_98083) = 320.0242
X_(98,_98095) = 712.7046
X_(98,_98099) = 292.37116
X_(99,_1197) = 399.16908
X_(99,_126286) = 551.12833
X_(99,_126724) = 649.39033
X_(99,_126729) =

In [ ]:
volumes_moved = [i.varValue for i in prob.variables()]


In [156]:
total_volume = 0
for i in range(0, len(volumes_moved)):
    total_volume = total_volume + volumes_moved[i]
print(total_volume)

59638912.490185946


In [ ]:
volume_matrix = []
k=0
while k < n_suppliers:
    vols = []
    for i in range(n_receivers):
        j = i + k*n_receivers
        vols.append(volumes_moved[j])
    volume_matrix.append(vols)
    k+=1

In [159]:
print(len(volume_matrix))
print(len(volume_matrix[0]))

289
158064


In [160]:
print(len(distance_matrix))
print(len(distance_matrix[0]))

289
158064


In [ ]:
results = []
for i in range(n_suppliers):
    f_loc = facility_latlon[i]
    for j in range(n_receivers):
        vol = volume_matrix[i][j]
        if vol > 0:
            w_loc = wells_latlon[j]
            dist = distance_matrix[i][j]
            r = [f_loc, w_loc, dist, vol]
            results.append(r)

In [ ]:
file_name = 'data/results/results_toydata.pkl'
open_file = open(file_name, "wb")
pickle.dump(results, open_file)
open_file.close()

In [ ]:
## USE THIS CODE TO IMPORT DATA

file_name = 'data/results/results_toydata.pkl'
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

